In [1]:
import pandas as pd
import requests
import os
import re
from pyzotero import zotero

In [2]:
!pwd

/Volumes/ssd/01-ckj-postdoc/LLM-CCR/boolean-search


In [3]:
emp_df = pd.read_csv('acl_empathy_related_papers.csv')
emo_df = pd.read_csv('acl_emotion_related_papers.csv')

In [4]:
emp_df.head()

,title,author,booktitle,year,url,abstract,title-abstract
0,Improved Text Emotion Prediction Using Combine...,"Mitsios, Michail and\nVamvoukakis, Georgios ...",Proceedings of the 2024 Conference of the Nort...,2024,https://aclanthology.org/2024.naacl-short.72,Emotion detection in textual data has received...,Improved Text Emotion Prediction Using Combine...
1,Modeling Empathetic Alignment in Conversation,"Yang, Jiamin and\nJurgens, David",Proceedings of the 2024 Conference of the Nort...,2024,https://aclanthology.org/2024.naacl-long.172,Empathy requires perspective-taking: empatheti...,Modeling Empathetic Alignment in Conversation ...
2,{A}cn{E}mpathize: A Dataset for Understanding ...,"Lee, Gyeongeun and\nParde, Natalie",Proceedings of the 2024 Joint International Co...,2024,https://aclanthology.org/2024.lrec-main.13,Empathy is critical for effective communicatio...,{A}cn{E}mpathize: A Dataset for Understanding ...
3,Appraisal Framework for Clinical Empathy: A No...,"Lahnala, Allison Claire and\nNeuendorf, B{\'e...",Proceedings of the 2024 Joint International Co...,2024,https://aclanthology.org/2024.lrec-main.124,Empathy is essential in healthcare communicati...,Appraisal Framework for Clinical Empathy: A No...
4,{ASEM}: Enhancing Empathy in Chatbot through A...,"Hamad, Omama and\nShaban, Khaled and\nHamdi,...",Proceedings of the 2024 Joint International Co...,2024,https://aclanthology.org/2024.lrec-main.140,Effective feature representations play a criti...,{ASEM}: Enhancing Empathy in Chatbot through A...


In [5]:
emo_df.head()

,title,author,booktitle,year,url,abstract,title-abstract
0,Estimating the Emotion of Disgust in {G}reek P...,"Lislevand, Vanessa and\nPavlopoulos, John an...",Proceedings of the 8th Workshop on Online Abus...,2024,https://aclanthology.org/2024.woah-1.9,We present an analysis of the sentiment in Gre...,Estimating the Emotion of Disgust in {G}reek P...
1,Topic Bias in Emotion Classification,"Wegge, Maximilian and\nKlinger, Roman",Proceedings of the Ninth Workshop on Noisy and...,2024,https://aclanthology.org/2024.wnut-1.9,Emotion corpora are typically sampled based on...,Topic Bias in Emotion Classification Emotion c...
2,{E}mo{M}ix-3{L}: A Code-Mixed Dataset for {B}a...,"Raihan, Nishat and\nGoswami, Dhiman and\nMah...",Proceedings of the 7th Workshop on Indian Lang...,2024,https://aclanthology.org/2024.wildre-1.2,Code-mixing is a well-studied linguistic pheno...,{E}mo{M}ix-3{L}: A Code-Mixed Dataset for {B}a...
3,"Ice and Fire: Dataset on Sentiment, Emotions, ...","Fri{\dh}riksd{\'o}ttir, Steinunn Rut and\nSim...","Proceedings of the Fourth Workshop on Threat, ...",2024,https://aclanthology.org/2024.trac-1.9,"This study introduces {``}Ice and Fire,{''} a ...","Ice and Fire: Dataset on Sentiment, Emotions, ..."
4,"Offensiveness, Hate, Emotion and {GPT}: Benchm...","Bauer, Nikolaj and\nPreisig, Moritz and\nVol...","Proceedings of the Fourth Workshop on Threat, ...",2024,https://aclanthology.org/2024.trac-1.14,"In this paper, we extend the work of benchmark...","Offensiveness, Hate, Emotion and {GPT}: Benchm..."


In [6]:
api_key = 'GvRPpGgrr2XyATxUdVhA2AG5'

In [7]:
ZOTERO_API_KEY = 'GvRPpGgrr2XyATxUdVhA2AG5'
ZOTERO_LIBRARY_ID = '11521923'
ZOTERO_LIBRARY_TYPE = 'user'

In [8]:
zot = zotero.Zotero(ZOTERO_LIBRARY_ID, ZOTERO_LIBRARY_TYPE, ZOTERO_API_KEY)

In [9]:
EMP_PDF_SAVE_PATH = 'ACL-EMPATHY'
os.makedirs(EMP_PDF_SAVE_PATH, exist_ok=True)

In [10]:
EMO_PDF_SAVE_PATH = 'ACL-EMOTION'
os.makedirs(EMO_PDF_SAVE_PATH, exist_ok=True)

In [11]:
def clean_filename(title):
    return re.sub(r'[^A-Za-z0-9]+', '_', title)

In [12]:
for index, row in emp_df.iterrows():
    url = row['url']
    title = row['title']
    clean_title = clean_filename(title)
    try:
        print(f"Attempting to download: {title} from {url}")
        pdf_response = requests.get(url)
        if pdf_response.status_code == 200:
            pdf_filename = os.path.join(EMP_PDF_SAVE_PATH, f"{clean_title}.pdf")
            with open(pdf_filename, 'wb') as pdf_file:
                pdf_file.write(pdf_response.content)
            print(f"Downloaded {title} successfully")

            item = {
                'itemType': 'journalArticle',
                'title': title,
                'url': url,
                'collections': ['UVBUPT6S'],
                'tags': [{'tag': 'Auto Downloaded'}, {'tag': 'Python Script'}]
            }
            created_items = zot.create_items([item])
            item_key = list(created_items['successful'].values())[0]['key']
            print(f"Created Zotero item for {title} with key {item_key}")

        else:
            print(f"Failed to download {title}: HTTP {pdf_response.status_code}")

    except Exception as e:
        print(f"Failed to download or add {title}: {e}")

Attempting to download: Improved Text Emotion Prediction Using Combined Valence and Arousal Ordinal Classification from https://aclanthology.org/2024.naacl-short.72
Downloaded Improved Text Emotion Prediction Using Combined Valence and Arousal Ordinal Classification successfully
Created Zotero item for Improved Text Emotion Prediction Using Combined Valence and Arousal Ordinal Classification with key SUJTJAED
Attempting to download: Modeling Empathetic Alignment in Conversation from https://aclanthology.org/2024.naacl-long.172
Downloaded Modeling Empathetic Alignment in Conversation successfully
Created Zotero item for Modeling Empathetic Alignment in Conversation with key 6QJFTW5P
Attempting to download: {A}cn{E}mpathize: A Dataset for Understanding Empathy in Dermatology Conversations from https://aclanthology.org/2024.lrec-main.13
Downloaded {A}cn{E}mpathize: A Dataset for Understanding Empathy in Dermatology Conversations successfully
Created Zotero item for {A}cn{E}mpathize: A Dat

In [13]:
for index, row in emo_df.iterrows():
    url = row['url']
    title = row['title']
    clean_title = clean_filename(title)
    try:
        print(f"Attempting to download: {title} from {url}")
        pdf_response = requests.get(url)
        if pdf_response.status_code == 200:
            pdf_filename = os.path.join(EMO_PDF_SAVE_PATH, f"{clean_title}.pdf")
            with open(pdf_filename, 'wb') as pdf_file:
                pdf_file.write(pdf_response.content)
            print(f"Downloaded {title} successfully")

            item = {
                'itemType': 'journalArticle',
                'title': title,
                'url': url,
                'collections': ['EZCWNT3L'],
                'tags': [{'tag': 'Auto Downloaded'}, {'tag': 'Python Script'}]
            }
            created_items = zot.create_items([item])
            item_key = list(created_items['successful'].values())[0]['key']
            print(f"Created Zotero item for {title} with key {item_key}")

        else:
            print(f"Failed to download {title}: HTTP {pdf_response.status_code}")

    except Exception as e:
        print(f"Failed to download or add {title}: {e}")

Attempting to download: Estimating the Emotion of Disgust in {G}reek Parliament Records from https://aclanthology.org/2024.woah-1.9
Downloaded Estimating the Emotion of Disgust in {G}reek Parliament Records successfully
Created Zotero item for Estimating the Emotion of Disgust in {G}reek Parliament Records with key VFZZ8SDW
Attempting to download: Topic Bias in Emotion Classification from https://aclanthology.org/2024.wnut-1.9
Downloaded Topic Bias in Emotion Classification successfully
Created Zotero item for Topic Bias in Emotion Classification with key 3ACZBFMD
Attempting to download: {E}mo{M}ix-3{L}: A Code-Mixed Dataset for {B}angla-{E}nglish-{H}indi for Emotion Detection from https://aclanthology.org/2024.wildre-1.2
Downloaded {E}mo{M}ix-3{L}: A Code-Mixed Dataset for {B}angla-{E}nglish-{H}indi for Emotion Detection successfully
Created Zotero item for {E}mo{M}ix-3{L}: A Code-Mixed Dataset for {B}angla-{E}nglish-{H}indi for Emotion Detection with key RZP68GTB
Attempting to downlo